In [1]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Si ves una lista con al menos un dispositivo GPU, TensorFlow la está detectando.
# Si la lista está vacía ([]), TensorFlow no está usando la GPU.
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))

GPUs disponibles: []


Importación de librerias.

In [ ]:
(imagenes_entrenamiento, etiquetas_entrenamiento ), (imagenes_verificacion, etiquetas_verificacion) = keras.datasets.cifar10.load_data()


Carga de conjunto de datos de imagenes para entrenamiento. 50.000 fotos en color de 32x32 pixels de aviones, trenes, autos, etc con etiquetas. 10000 fotos para testear

In [ ]:
print(etiquetas_entrenamiento.shape, imagenes_entrenamiento.shape)

In [ ]:
imagenes_entrenamiento, imagenes_verificacion = imagenes_entrenamiento / 255.0, imagenes_verificacion  / 255.0

AGREGADO: Estandarizacion entre 0s y 1s

In [ ]:
nombres_clases = ['airplane','automobile','bird','cat','deer',
                  'dog','frog','horse','ship','truck']

Funcion para mostrar imagenes en miniatura.

In [ ]:
import matplotlib.pyplot as plt
def mostrar():
  plt.figure(figsize=(5,5))
  for i in range(10):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(imagenes_entrenamiento[i], cmap=plt.cm.binary)
    plt.xlabel(nombres_clases[etiquetas_entrenamiento[i][0]])
  plt.show()
mostrar()

Una capa de entrada para imagenes de 32x32 pixels con 3 dimensiones de colores.
Dos capas convolucionales para extraer caracteristicas, con 32 filtros de 3x3. Quitando relleno que no se ajusta.
Dos capas de agrupamiento con filtros de 2x2 para achicar la anterior.

Una capa complemtamente conectada (densa) con 64 neuronas.
Una capa de salida con 10 neuronas para clasificar.

In [ ]:
capa = keras.layers
modelo = keras.models.Sequential()
modelo.add(capa.Conv2D(32, (3,3), strides=(1,1), padding="valid", activation='relu',
                       input_shape=(32,32,3)))
modelo.add(capa.MaxPool2D(2,2))
modelo.add(capa.Conv2D(32, 3, activation='relu' ))
modelo.add(capa.MaxPool2D(2,2))
modelo.add(capa.Flatten())

modelo.add(capa.Dense(64, activation='relu'))
modelo.add(capa.Dense(10))

modelo.summary()  #(3x3xC+1)x32 = 896 (C=Canales) 32 filtros. kernel 3x3

Configuracion del modelo

In [ ]:
modelo.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

Entrenamiento (creación del modelo)

In [ ]:
modelo.fit(imagenes_entrenamiento, etiquetas_entrenamiento,
           epochs=3, batch_size=64)

Loss se busca el número mas bajo. Accuracy el más alto

In [ ]:
modelo.evaluate(imagenes_verificacion, etiquetas_verificacion,
                batch_size=64)

In [ ]:
#predict()

**CONTINUAR DESDE ACA**